# Test add_sub_reverse_proxy

In [ ]:
from fastcaddy.core import *

In [ ]:
pcfg({})

[2025-02-19 17:52:59 - httpx:1025 - INFO] HTTP Request: POST http://localhost:2019/config/ "HTTP/1.1 200 OK"


In [ ]:
cf_token = 'DUMMY_TOKEN'

In [ ]:
setup_caddy(cf_token)

[2025-02-19 18:52:05 - httpx:1025 - INFO] HTTP Request: GET http://localhost:2019/config/apps/tls/automation/ "HTTP/1.1 200 OK"
[2025-02-19 18:52:05 - httpx:1025 - INFO] HTTP Request: GET http://localhost:2019/config/apps/http/servers/ "HTTP/1.1 200 OK"


At this point our Caddy config is:

In [ ]:
{
  "apps": {
    "http": {
      "servers": {
        "srv0": {
          "listen": [
            ":80",
            ":443"
          ],
          "routes": []
        }
      }
    },
    "tls": {
      "automation": {
        "policies": [
          {
            "issuers": [
              {
                "challenges": {
                  "dns": {
                    "provider": {
                      "api_token": "XXX",
                      "name": "cloudflare"
                    }
                  }
                },
                "module": "acme"
              }
            ]
          }
        ]
      }
    }
  }
}

{'apps': {'http': {'servers': {'srv0': {'listen': [':80', ':443'],
     'routes': []}}},
  'tls': {'automation': {'policies': [{'issuers': [{'challenges': {'dns': {'provider': {'api_token': 'XXX',
           'name': 'cloudflare'}}},
        'module': 'acme'}]}]}}}}

## Setup: add wildcard route *.something.example.com. 

This is needed in order to add subroutes to it. 

In [ ]:
add_wildcard_route('something.example.com')

[2025-02-19 17:56:19 - httpx:1025 - INFO] HTTP Request: POST http://localhost:2019/config/apps/http/servers/srv0/routes/ "HTTP/1.1 200 OK"


Now our config should include the wildcard route:

In [ ]:
import json
from rich import print

In [ ]:
def print_json(j): print(json.dumps(j))

In [ ]:
d = {
  "apps": {
    "http": {
      "servers": {
        "srv0": {
          "listen": [
            ":80",
            ":443"
          ],
          "routes": [
            {
              "@id": "wildcard-something.example.com",
              "handle": [
                {
                  "handler": "subroute",
                  "routes": []
                }
              ],
              "match": [
                {
                  "host": [
                    "*.something.example.com"
                  ]
                }
              ],
              "terminal": True
            }
          ]
        }
      }
    },
    "tls": {
      "automation": {
        "policies": [
          {
            "issuers": [
              {
                "challenges": {
                  "dns": {
                    "provider": {
                      "api_token": "XXX",
                      "name": "cloudflare"
                    }
                  }
                },
                "module": "acme"
              }
            ]
          }
        ]
      }
    }
  }
}
print_json(d)

{"apps": {"http": {"servers": {"srv0": {"listen": [":80", ":443"], "routes": [{"@id": 
"wildcard-something.example.com", "handle": [{"handler": "subroute", "routes": []}], "match": [{"host": 
["*.something.example.com"]}], "terminal": true}]}}}, "tls": {"automation": {"policies": [{"issuers": 
[{"challenges": {"dns": {"provider": {"api_token": "XXX", "name": "cloudflare"}}}, "module": "acme"}]}]}}}}

At this point there are no subroutes associated with that `*.something.example.com` wildcard. When that is matched, the handlers list is empty until...

## Add subroute 1: foo.something.example.com 

In [ ]:
add_sub_reverse_proxy('something.example.com', 'foo', 5001)

[2025-02-19 18:05:21 - httpx:1025 - INFO] HTTP Request: POST http://localhost:2019/id/wildcard-something.example.com/handle/0/routes/.../ "HTTP/1.1 200 OK"


In [ ]:
gcfg('/apps/http/servers/srv0/routes/0/handle')

[2025-02-19 18:09:58 - httpx:1025 - INFO] HTTP Request: GET http://localhost:2019/config/apps/http/servers/srv0/routes/0/handle/ "HTTP/1.1 200 OK"


(#1) [{'handler': 'subroute', 'routes': [{'@id': 'foo.something.example.com', 'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5001'}]}], 'match': [{'host': ['foo.something.example.com']}]}]}]

Now we can see the `handle` config with the list of sub-routes and the sub-route handler. 

In [ ]:
gcfg('/apps/http/servers/srv0/routes/0/handle/0/routes')

[2025-02-19 18:11:31 - httpx:1025 - INFO] HTTP Request: GET http://localhost:2019/config/apps/http/servers/srv0/routes/0/handle/0/routes/ "HTTP/1.1 200 OK"


(#1) [{'@id': 'foo.something.example.com', 'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5001'}]}], 'match': [{'host': ['foo.something.example.com']}]}]

We can see that:

* A route with id `foo.something.example.com` was created. 
* It matches requests for `foo.something.example.com`
* When a request for `foo.something.example.com` is matched, its handler is a reverse proxy to `localhost:5001`. 

## Add subroute 2: bar.something.example.com 

In [ ]:
add_sub_reverse_proxy('something.example.com', 'bar', 5002)

[2025-02-19 18:15:28 - httpx:1025 - INFO] HTTP Request: POST http://localhost:2019/id/wildcard-something.example.com/handle/0/routes/.../ "HTTP/1.1 200 OK"


In [ ]:
gcfg('/apps/http/servers/srv0/routes/0/handle')

[2025-02-19 18:15:37 - httpx:1025 - INFO] HTTP Request: GET http://localhost:2019/config/apps/http/servers/srv0/routes/0/handle/ "HTTP/1.1 200 OK"


(#1) [{'handler': 'subroute', 'routes': [{'@id': 'foo.something.example.com', 'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5001'}]}], 'match': [{'host': ['foo.something.example.com']}]}, {'@id': 'bar.something.example.com', 'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5002'}]}], 'match': [{'host': ['bar.something.example.com']}]}]}]

Now we see a single sub-route handler containing two routes. 

In [ ]:
gcfg('/apps/http/servers/srv0/routes/0/handle/0/routes')

[2025-02-19 18:16:33 - httpx:1025 - INFO] HTTP Request: GET http://localhost:2019/config/apps/http/servers/srv0/routes/0/handle/0/routes/ "HTTP/1.1 200 OK"


(#2) [{'@id': 'foo.something.example.com', 'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5001'}]}], 'match': [{'host': ['foo.something.example.com']}]},{'@id': 'bar.something.example.com', 'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5002'}]}], 'match': [{'host': ['bar.something.example.com']}]}]

Here we see that:

* The `foo.something.example.com` subroute is still present. 
* A route with id `bar.something.example.com` was created. 
* It matches requests for `bar.something.example.com`
* When a request for `bar.something.example.com` is matched, its handler is a reverse proxy to `localhost:5002`. 

In [ ]:
%%ai
Add nbdev tests for each of the above

I'll help create nbdev tests for the functionality demonstrated above. We'll test the sequence of operations: setting up Caddy, adding a wildcard route, and adding subroutes.

Here's a test module we can create:

```python
# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/03_wildcard_routes.ipynb

__all__ = []

# %% ../nbs/03_wildcard_routes.ipynb 1
import pytest
from fastcaddy.core import *
from fastcaddy.testing import *

# %% ../nbs/03_wildcard_routes.ipynb 2
@pytest.fixture(scope="module")
def setup():
    """Setup fresh Caddy config for testing"""
    pcfg({})
    setup_caddy('DUMMY_TOKEN')
    
# %% ../nbs/03_wildcard_routes.ipynb 3
def test_wildcard_route(setup):
    """Test adding a wildcard route for a domain"""
    add_wildcard_route('something.example.com')
    
    # Verify route was added correctly
    routes = gcfg('/apps/http/servers/srv0/routes')
    assert len(routes) == 1
    route = routes[0]
    
    # Check route properties
    assert route['@id'] == 'wildcard-something.example.com'
    assert route['terminal'] == True
    assert route['match'][0]['host'] == ['*.something.example.com']
    assert route['handle'][0]['handler'] == 'subroute'
    assert route['handle'][0]['routes'] == []

# %% ../nbs/03_wildcard_routes.ipynb 4
def test_add_subroutes(setup):
    """Test adding multiple subroutes to a wildcard route"""
    # Setup wildcard route first
    add_wildcard_route('something.example.com')
    
    # Add first subroute
    add_sub_reverse_proxy('something.example.com', 'foo', 5001)
    
    # Verify first subroute
    routes = gcfg('/apps/http/servers/srv0/routes/0/handle/0/routes')
    assert len(routes) == 1
    assert routes[0]['@id'] == 'foo.something.example.com'
    assert routes[0]['match'][0]['host'] == ['foo.something.example.com']
    assert routes[0]['handle'][0]['handler'] == 'reverse_proxy'
    assert routes[0]['handle'][0]['upstreams'][0]['dial'] == 'localhost:5001'
    
    # Add second subroute
    add_sub_reverse_proxy('something.example.com', 'bar', 5002)
    
    # Verify both subroutes
    routes = gcfg('/apps/http/servers/srv0/routes/0/handle/0/routes')
    assert len(routes) == 2
    
    # Verify second subroute
    assert routes[1]['@id'] == 'bar.something.example.com'
    assert routes[1]['match'][0]['host'] == ['bar.something.example.com']
    assert routes[1]['handle'][0]['handler'] == 'reverse_proxy'
    assert routes[1]['handle'][0]['upstreams'][0]['dial'] == 'localhost:5002'
```

These tests cover:

1. Basic setup with a fixture that resets the Caddy config before each test
2. Adding a wildcard route and verifying its structure
3. Adding multiple subroutes and verifying their configuration

Would you like me to explain any part in more detail or add additional test cases?